# Flight_Price_Prediction

In [62]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing Important libraries

In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score


In [64]:
sns.set(style='darkgrid')
rcParams['figure.figsize']=10,6

In [65]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [66]:
train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,Airline C,12/06/2019,Delhi,Cochin,DEL → MAA → COK,20:40,09:25 13 Jun,12h 45m,1 stop,No info,7480
1,Airline A,18/06/2019,Banglore,Delhi,BLR → DEL,18:55,22:00,3h 5m,non-stop,No info,8016
2,Airline C,18/05/2019,Delhi,Cochin,DEL → BOM → COK,03:50,19:15,15h 25m,1 stop,No info,8879
3,Airline A,6/05/2019,Kolkata,Banglore,CCU → BOM → BLR,20:00,08:15 07 May,12h 15m,1 stop,In-flight meal not included,9663
4,Airline A,9/05/2019,Kolkata,Banglore,CCU → BOM → BLR,06:30,12:00,5h 30m,1 stop,In-flight meal not included,9663


In [67]:
test.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Airline B,27-03-2019,Delhi,Cochin,DEL → HYD → COK,04:55,16:10,11h 15m,1 stop,No info
1,Airline E,27-05-2019,Kolkata,Banglore,CCU → BLR,22:20,28-05-2020 00:40,2h 20m,non-stop,No info
2,Airline C,06-06-2019,Kolkata,Banglore,CCU → IXR → DEL → BLR,05:50,20:25,14h 35m,2 stops,No info
3,Airline A,06-03-2019,Banglore,New Delhi,BLR → MAA → DEL,09:45,14:25,4h 40m,1 stop,No info
4,Airline B,15-06-2019,Delhi,Cochin,DEL → BOM → COK,16:00,16-06-2020 01:30,9h 30m,1 stop,No info


In [68]:
train.isna().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [69]:
test.isna().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
dtype: int64

In [70]:
train.shape, test.shape


((8012, 11), (2671, 10))

In [71]:
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8012 entries, 0 to 8011
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          8012 non-null   object
 1   Date_of_Journey  8012 non-null   object
 2   Source           8012 non-null   object
 3   Destination      8012 non-null   object
 4   Route            8011 non-null   object
 5   Dep_Time         8012 non-null   object
 6   Arrival_Time     8012 non-null   object
 7   Duration         8012 non-null   object
 8   Total_Stops      8011 non-null   object
 9   Additional_Info  8012 non-null   object
 10  Price            8012 non-null   int64 
dtypes: int64(1), object(10)
memory usage: 688.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          2671 non-null   object
 1   Date_of_

(None, None)

In [72]:
train.dropna(inplace=True)

In [73]:
train.shape

(8011, 11)

In [74]:
train['jou_day'] = pd.DatetimeIndex(train['Date_of_Journey']).day
train['jou_mon'] = pd.DatetimeIndex(train['Date_of_Journey']).month

In [75]:
test['jou_day'] = pd.DatetimeIndex(test['Date_of_Journey']).day
test['jou_mon'] = pd.DatetimeIndex(test['Date_of_Journey']).month

## catrgorical features

In [76]:
train['Airline'].value_counts()


Airline A    2881
Airline B    1550
Airline C    1286
Airline D     906
Airline E     625
Airline F     360
Airline G     235
Airline H     146
Airline I      12
Airline J       6
Airline K       3
Airline L       1
Name: Airline, dtype: int64

In [77]:
test['Airline'].value_counts()

Airline A    968
Airline B    503
Airline C    465
Airline D    290
Airline E    193
Airline F    119
Airline G     84
Airline H     48
Airline I      1
Name: Airline, dtype: int64

In [78]:
train['Destination'].unique()


array(['Cochin', 'Delhi', 'Banglore', 'New Delhi', 'Hyderabad', 'Kolkata'],
      dtype=object)

In [79]:
test['Destination'].unique()

array(['Cochin', 'Banglore', 'New Delhi', 'Hyderabad', 'Delhi', 'Kolkata'],
      dtype=object)

In [80]:
train_airline=pd.get_dummies(train.Airline,drop_first=True)


In [81]:
train['Airline'].value_counts()

Airline A    2881
Airline B    1550
Airline C    1286
Airline D     906
Airline E     625
Airline F     360
Airline G     235
Airline H     146
Airline I      12
Airline J       6
Airline K       3
Airline L       1
Name: Airline, dtype: int64

In [82]:
test['Airline'].value_counts()

Airline A    968
Airline B    503
Airline C    465
Airline D    290
Airline E    193
Airline F    119
Airline G     84
Airline H     48
Airline I      1
Name: Airline, dtype: int64

In [83]:
train_airline=train_airline.drop(['Airline J','Airline K', 'Airline L' ],axis=1)

In [84]:
train_airline.head()

,Airline B,Airline C,Airline D,Airline E,Airline F,Airline G,Airline H,Airline I
0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0


In [85]:
test_airline=pd.get_dummies(test.Airline,drop_first=True)


In [86]:
test_airline.head()

,Airline B,Airline C,Airline D,Airline E,Airline F,Airline G,Airline H,Airline I
0,1,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0
2,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0


In [87]:
train_source=pd.get_dummies(train.Source,drop_first=True)
print(train_source.head())

train_destination=pd.get_dummies(train.Destination,drop_first=True)
print(train_destination.head())

   Chennai  Delhi  Kolkata  Mumbai
0        0      1        0       0
1        0      0        0       0
2        0      1        0       0
3        0      0        1       0
4        0      0        1       0
   Cochin  Delhi  Hyderabad  Kolkata  New Delhi
0       1      0          0        0          0
1       0      1          0        0          0
2       1      0          0        0          0
3       0      0          0        0          0
4       0      0          0        0          0


In [88]:
test_source=pd.get_dummies(test.Source,drop_first=True)
print(test_source.head())

test_destination=pd.get_dummies(test.Destination,drop_first=True)
print(test_destination.head())

   Chennai  Delhi  Kolkata  Mumbai
0        0      1        0       0
1        0      0        1       0
2        0      0        1       0
3        0      0        0       0
4        0      1        0       0
   Cochin  Delhi  Hyderabad  Kolkata  New Delhi
0       1      0          0        0          0
1       0      0          0        0          0
2       0      0          0        0          0
3       0      0          0        0          1
4       1      0          0        0          0


In [89]:
train['Total_Stops'].unique()


array(['1 stop', 'non-stop', '2 stops', '3 stops'], dtype=object)

In [90]:
stops={'non-stop':0,'1 stop':1,'2 stops':2,'3 stops':3,'4 stops':4}

train['Total_Stops']=train['Total_Stops'].map(stops)

In [91]:
train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,jou_day,jou_mon
0,Airline C,12/06/2019,Delhi,Cochin,DEL → MAA → COK,20:40,09:25 13 Jun,12h 45m,1,No info,7480,6,12
1,Airline A,18/06/2019,Banglore,Delhi,BLR → DEL,18:55,22:00,3h 5m,0,No info,8016,18,6
2,Airline C,18/05/2019,Delhi,Cochin,DEL → BOM → COK,03:50,19:15,15h 25m,1,No info,8879,18,5
3,Airline A,6/05/2019,Kolkata,Banglore,CCU → BOM → BLR,20:00,08:15 07 May,12h 15m,1,In-flight meal not included,9663,5,6
4,Airline A,9/05/2019,Kolkata,Banglore,CCU → BOM → BLR,06:30,12:00,5h 30m,1,In-flight meal not included,9663,5,9


In [92]:
test['Total_Stops'].unique()

array(['1 stop', 'non-stop', '2 stops', '3 stops', '4 stops'],
      dtype=object)

In [93]:
stops={'non-stop':0,'1 stop':1,'2 stops':2,'3 stops':3,'4 stops':4}

test['Total_Stops']=test['Total_Stops'].map(stops)


In [94]:
train['dep_hour']=pd.to_datetime(train['Dep_Time']).dt.hour
train['dep_min']=pd.to_datetime(train['Dep_Time']).dt.minute

In [95]:
test['dep_hour']=pd.to_datetime(test['Dep_Time']).dt.hour
test['dep_min']=pd.to_datetime(test['Dep_Time']).dt.minute

In [96]:
train['arr_hour']=pd.to_datetime(train['Arrival_Time']).dt.hour
train['arr_min']=pd.to_datetime(train['Arrival_Time']).dt.minute

In [97]:
test['arr_hour']=pd.to_datetime(test['Arrival_Time']).dt.hour
test['arr_min']=pd.to_datetime(test['Arrival_Time']).dt.minute

In [98]:
train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,jou_day,jou_mon,dep_hour,dep_min,arr_hour,arr_min
0,Airline C,12/06/2019,Delhi,Cochin,DEL → MAA → COK,20:40,09:25 13 Jun,12h 45m,1,No info,7480,6,12,20,40,9,25
1,Airline A,18/06/2019,Banglore,Delhi,BLR → DEL,18:55,22:00,3h 5m,0,No info,8016,18,6,18,55,22,0
2,Airline C,18/05/2019,Delhi,Cochin,DEL → BOM → COK,03:50,19:15,15h 25m,1,No info,8879,18,5,3,50,19,15
3,Airline A,6/05/2019,Kolkata,Banglore,CCU → BOM → BLR,20:00,08:15 07 May,12h 15m,1,In-flight meal not included,9663,5,6,20,0,8,15
4,Airline A,9/05/2019,Kolkata,Banglore,CCU → BOM → BLR,06:30,12:00,5h 30m,1,In-flight meal not included,9663,5,9,6,30,12,0


In [99]:
train.drop(['Airline','Date_of_Journey','Source','Destination','Route','Dep_Time','Arrival_Time','Duration','Additional_Info'],axis=1,inplace=True)

In [100]:
test.drop(['Airline','Date_of_Journey','Source','Destination','Route','Dep_Time','Arrival_Time','Duration','Additional_Info'],axis=1,inplace=True)

In [101]:
train.head()

,Total_Stops,Price,jou_day,jou_mon,dep_hour,dep_min,arr_hour,arr_min
0,1,7480,6,12,20,40,9,25
1,0,8016,18,6,18,55,22,0
2,1,8879,18,5,3,50,19,15
3,1,9663,5,6,20,0,8,15
4,1,9663,5,9,6,30,12,0


In [102]:
test.head()

,Total_Stops,jou_day,jou_mon,dep_hour,dep_min,arr_hour,arr_min
0,1,27,3,4,55,16,10
1,0,27,5,22,20,0,40
2,2,6,6,5,50,20,25
3,1,3,6,9,45,14,25
4,1,15,6,16,0,1,30


In [103]:
train=pd.concat([train,train_airline,train_source,train_destination],axis=1)

In [104]:
train.head()

,Total_Stops,Price,jou_day,jou_mon,dep_hour,dep_min,arr_hour,arr_min,Airline B,Airline C,...,Airline I,Chennai,Delhi,Kolkata,Mumbai,Cochin,Delhi,Hyderabad,Kolkata,New Delhi
0,1,7480,6,12,20,40,9,25,0,1,...,0,0,1,0,0,1,0,0,0,0
1,0,8016,18,6,18,55,22,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1,8879,18,5,3,50,19,15,0,1,...,0,0,1,0,0,1,0,0,0,0
3,1,9663,5,6,20,0,8,15,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,9663,5,9,6,30,12,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [105]:
test=pd.concat([test,test_airline,test_source,test_destination],axis=1)

In [106]:
test.head()

,Total_Stops,jou_day,jou_mon,dep_hour,dep_min,arr_hour,arr_min,Airline B,Airline C,Airline D,...,Airline I,Chennai,Delhi,Kolkata,Mumbai,Cochin,Delhi,Hyderabad,Kolkata,New Delhi
0,1,27,3,4,55,16,10,1,0,0,...,0,0,1,0,0,1,0,0,0,0
1,0,27,5,22,20,0,40,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2,6,6,5,50,20,25,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,1,3,6,9,45,14,25,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,15,6,16,0,1,30,1,0,0,...,0,0,1,0,0,1,0,0,0,0


In [107]:
train.shape, test.shape

((8011, 25), (2671, 24))

In [108]:
X=train.drop(['Price'],axis=1)
y=train['Price']

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=5)

## KNeighborsRegressor

In [110]:
knr=KNeighborsRegressor(n_neighbors=2)
knr.fit(X_train,y_train)
y_pred=knr.predict(X_test)

In [111]:
score=r2_score(y_test,y_pred)
score

0.48539644737652443

## RandomForestRegressor

In [112]:
regressor=RandomForestRegressor(n_estimators = 100, random_state = 15)
regressor.fit(X_train,y_train)
y_Pred=regressor.predict(X_test)

In [113]:
scores=r2_score(y_test,y_Pred)
scores

0.7962726770587041

In [114]:
-np.sqrt(np.square(np.log10(y_pred +1) - np.log10(y_test +1)).mean())

-0.14233359692338154

In [115]:
train_pred=regressor.predict(test)
final_df=pd.DataFrame({ 'Price': train_pred })
final_df.to_csv('final_dataset.csv',index=False)

In [116]:
y_pred=knr.predict(test)
final_df=pd.DataFrame({ 'Price': y_pred })
final_df.to_csv('final_dataset_knr.csv',index=False)